#### This notebook details the complete data preprocessing and merging procedures, culminating in the statistics reported in Table 1. Thsi code also  generates  'df_flu_with_flu_rate.csv' and 'df_merged.csv' for the follwoing analysis 

##  Cities

In [1]:
import os
import geopandas as gpd
import pandas as pd

# 读取GeoJSON文件
gdf = gpd.read_file("E:\\sleep cycle\\Geodata\\selected_geometry_all.geojson")


# 2. 设置数据目录
data_dir = r"E:\sleep cycle\data\cough_rate_with_user_count"

# 3. 初始化字典收集所有读取的数据
data_dict = {}

results = []

# 4. 遍历所有 NAME_2
for name in gdf['NAME_2'].unique():
    if pd.isna(name):
        continue  # 跳过缺失值
    
    filename = "cough_aggregates_" + name.replace(" ", "_") + "_2024.csv"
    filepath = os.path.join(data_dir, filename)
    
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        
        avg_hours = df['total_session_hours'].mean()
        avg_users = df['total_users'].mean()
        avg_cough_rate = df['cough_rate'].mean()
        total_days = len(df)
        results.append({"City": name, "avg_hours": avg_hours, "avg_users":avg_users, "avg_cough_rate":avg_cough_rate,"total_days": total_days})
            
        data_dict[name] = df
    else:
        print(f"⚠️ 文件不存在：{filepath}")

# 5. 打印成功读取的城市数量
print(f"\n成功读取 {len(data_dict)} 个城市的数据。")

# 5. 转为 DataFrame
df_total = pd.DataFrame(results)


成功读取 49 个城市的数据。


In [2]:
df_total

,City,avg_hours,avg_users,avg_cough_rate,total_days
0,Bangalore,551.854257,97.441584,0.647474,505
1,Beijing,8676.637157,1361.633333,0.558888,510
2,Bergamo,808.741818,122.652525,0.694822,495
3,Berlin,18642.766504,2796.610568,0.592418,511
4,Birmingham,4966.653718,754.761252,0.721695,511
5,Bogota,2105.928170,344.586275,0.630438,510
6,Brisbane,21801.531931,3207.896282,0.584968,511
7,Cambridgeshire,4289.139432,615.954990,0.677420,511
8,Chengdu,2479.888987,396.870588,0.560845,510
9,Chicago,26136.038542,3944.730469,0.577351,512


## Air quality

In [3]:
# 2. 设置数据目录
air_dir = r"E:\sleep cycle\data\air_quality_all"

# 3. 初始化字典收集所有读取的数据
data_dict = {}

results = []

# 4. 遍历所有 NAME_2
for name in gdf['NAME_2'].unique():
    if pd.isna(name):
        continue  # 跳过缺失值
    
    filename = name.replace(" ", "-") + "-air-quality.csv"
    filepath = os.path.join(air_dir, filename)
    
    if os.path.exists(filepath):
        
        df = pd.read_csv(filepath)
        df.columns = df.columns.str.strip()
        
        # 确保 date 是 datetime 类型
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['date'] >= '2023-10-01']
       
        df['pm25'] = pd.to_numeric(df['pm25'], errors='coerce')
        
        avg_PM25 = df['pm25'].mean()
        
        total_days = len(df)
        results.append({"City": name, "avg_PM25": avg_PM25,"air_total_days": total_days})
        
        data_dict[name] = df
    else:
        print(f"⚠️ 文件不存在：{filepath}")

# 5. 打印成功读取的城市数量
print(f"\n成功读取 {len(data_dict)} 个城市的数据。")

df_PM25 = pd.DataFrame(results)


成功读取 49 个城市的数据。


In [4]:
df_PM25

,City,avg_PM25,air_total_days
0,Bangalore,73.872495,582
1,Beijing,89.290271,628
2,Bergamo,13.920129,696
3,Berlin,52.151515,628
4,Birmingham,33.834798,573
5,Bogota,62.129323,674
6,Brisbane,25.243902,623
7,Cambridgeshire,4.975957,517
8,Chengdu,106.097756,626
9,Chicago,32.802885,625


## Temperature and precipitation

In [5]:
# 2. 设置数据目录
air_dir = r"E:\sleep cycle\data\weather_all"

# 3. 初始化字典收集所有读取的数据
data_dict = {}

results = []

# 4. 遍历所有 NAME_2
for name in gdf['NAME_2'].unique():
#     print(name)
    if pd.isna(name):
        continue  # 跳过缺失值
    
    filename = name + "_noaa.csv"
    filepath = os.path.join(air_dir, filename)
    
    if os.path.exists(filepath):
        
        df = pd.read_csv(filepath)
        df.columns = df.columns.str.strip()
        
        # 确保 date 是 datetime 类型
        df['date'] = pd.to_datetime(df['DATE'])

        # 过滤出 2023年10月1日（含）之后的数据
        df = df[df['date'] >= '2023-10-01']

        df['TAVG'] = pd.to_numeric(df['TAVG'], errors='coerce')
        df['PRCP'] = pd.to_numeric(df['PRCP'], errors='coerce')
        
        avg_TAVG = df['TAVG'].mean()
        avg_PRCP = df['PRCP'].mean()
        
        total_day = len(df)
        results.append({"City": name, "avg_TAVG": avg_TAVG, "avg_PRCP": avg_PRCP,"climate_total_days": total_day})
        
        data_dict[name] = df
    else:
        print(f"⚠️ 文件不存在：{filepath}")

# 5. 打印成功读取的城市数量
print(f"\n成功读取 {len(data_dict)} 个城市的数据。")

df_weather = pd.DataFrame(results)


成功读取 49 个城市的数据。


In [6]:
df_weather

,City,avg_TAVG,avg_PRCP,climate_total_days
0,Bangalore,76.256536,0.313450,612
1,Beijing,51.278481,0.064301,553
2,Bergamo,56.043928,0.146068,387
3,Berlin,49.536313,0.065505,548
4,Birmingham,50.412561,0.098029,548
5,Bogota,58.147147,0.898200,666
6,Brisbane,72.223833,0.172068,622
7,Cambridgeshire,56.038062,0.112895,578
8,Chengdu,60.422182,0.090475,559
9,Chicago,47.252019,0.082621,624


## Demographics

In [7]:
filepath = r'E:\sleep cycle\meta_data_countries_new.csv'
df_meta = pd.read_csv(filepath)

In [8]:
df_meta

,Tabell 1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,Male/female ratio,Age Mean,Age Std,Age Median,Age Q1(25%<),Age Q3 (75%<)
1,China,1.45,32.6,9.07,31,27,37
2,UK,1.23,37,13.06,34,26,46
3,United States,1.07,39.8,15,36,27,50
4,Italy,1.93,39.7,13.05,37,28,49
5,Sweden,1.14,38.31,14.91,36,25,52
6,India,2.5,40.5,13.07,38,30,51
7,South Africa,2,41.76,14.6,40,31,52
8,Japan,1.83,43,13.07,44,32,54
9,France,1.45,35.9,13.02,32,25,47


##  Merge all

In [9]:
# 先合并 df_PM25 和 df_total
df_merged = pd.merge(df_PM25, df_total, on='City', how='inner')

# 再合并 df_merged 和 df_wealth
df_merged = pd.merge(df_merged, df_weather, on='City', how='inner')

# 创建一个从 NAME_2 到 NAME_0 的映射（字典）
name2_to_name0 = gdf.dropna(subset=['NAME_2']).drop_duplicates(subset=['NAME_2'])[['NAME_2', 'NAME_0']]
name2_to_name0_dict = dict(zip(name2_to_name0['NAME_2'], name2_to_name0['NAME_0']))

df_merged['Country'] = df_merged['City'].map(name2_to_name0_dict)
df_merged['Country-city'] = df_merged['Country'] + '-' + df_merged['City']

# 按 country 和 city 排序
df_merged = df_merged.drop(columns=['City', 'Country'])
df_merged = df_merged.sort_values(by='Country-city').reset_index(drop=True)


In [10]:
df_merged

,avg_PM25,air_total_days,avg_hours,avg_users,avg_cough_rate,total_days,avg_TAVG,avg_PRCP,climate_total_days,Country-city
0,25.243902,623,21801.531931,3207.896282,0.584968,511,72.223833,0.172068,622,Australia-Brisbane
1,32.546493,613,32035.588813,4733.064579,0.568597,511,59.508065,0.081542,620,Australia-Melbourne
2,21.725678,628,33913.194162,5077.722114,0.584062,511,65.942717,0.100586,611,Australia-Sydney
3,51.156766,613,16205.574544,2610.390625,0.691427,512,70.055976,0.138514,661,Brazil-Sao Paulo
4,65.302885,630,3304.354395,529.849609,0.733482,512,60.225564,0.024577,665,Chile-Santiago
5,89.290271,628,8676.637157,1361.633333,0.558888,510,51.278481,0.064301,553,China-Beijing
6,106.097756,626,2479.888987,396.870588,0.560845,510,60.422182,0.090475,559,China-Chengdu
7,108.001603,626,22.741248,3.903766,0.359283,478,37.472039,0.047163,608,China-Daqing
8,80.978474,511,3103.370523,521.398039,0.597729,510,72.608541,0.191153,562,China-Guangzhou
9,91.641476,571,2816.383301,448.369352,0.531760,509,63.202341,0.170759,598,China-Hangzhou


In [11]:
df_population = pd.read_csv('City_Population_Data.csv')
df_merged = pd.merge(df_merged, df_population, on='Country-city', how='inner')


df_merged['user_rate'] = df_merged['avg_users']/df_merged['Population']*10000

In [12]:
df_population

,Country,City,Population,Country-city
0,Australia,Brisbane,2570000,Australia-Brisbane
1,Australia,Melbourne,5390000,Australia-Melbourne
2,Australia,Sydney,5560000,Australia-Sydney
3,China,Beijing,22596000,China-Beijing
4,China,Chengdu,7416000,China-Chengdu
5,China,Daqing,2090000,China-Daqing
6,China,Guangzhou,14879000,China-Guangzhou
7,China,Hangzhou,6242000,China-Hangzhou
8,China,Harbin,5879000,China-Harbin
9,China,Shanghai,30000000,China-Shanghai


In [13]:
# df_merged.to_csv('df_merged.csv', index=False, encoding='utf-8-sig')

In [14]:
# df_merged = df_merged[df_merged['avg_users'] > 1000].reset_index(drop=True)
df_merged = df_merged[df_merged['user_rate'] > 0.5].reset_index(drop=True)
df_merged

,avg_PM25,air_total_days,avg_hours,avg_users,avg_cough_rate,total_days,avg_TAVG,avg_PRCP,climate_total_days,Country-city,Country,City,Population,user_rate
0,25.243902,623,21801.531931,3207.896282,0.584968,511,72.223833,0.172068,622,Australia-Brisbane,Australia,Brisbane,2570000,12.482087
1,32.546493,613,32035.588813,4733.064579,0.568597,511,59.508065,0.081542,620,Australia-Melbourne,Australia,Melbourne,5390000,8.781196
2,21.725678,628,33913.194162,5077.722114,0.584062,511,65.942717,0.100586,611,Australia-Sydney,Australia,Sydney,5560000,9.132594
3,51.156766,613,16205.574544,2610.390625,0.691427,512,70.055976,0.138514,661,Brazil-Sao Paulo,Brazil,Sao Paulo,12000000,2.175326
4,65.302885,630,3304.354395,529.849609,0.733482,512,60.225564,0.024577,665,Chile-Santiago,Chile,Santiago,6999000,0.757036
5,89.290271,628,8676.637157,1361.633333,0.558888,510,51.278481,0.064301,553,China-Beijing,China,Beijing,22596000,0.602599
6,106.097756,626,2479.888987,396.870588,0.560845,510,60.422182,0.090475,559,China-Chengdu,China,Chengdu,7416000,0.535155
7,91.641476,571,2816.383301,448.369352,0.531760,509,63.202341,0.170759,598,China-Hangzhou,China,Hangzhou,6242000,0.718310
8,89.940989,628,11831.514379,1823.125490,0.555558,510,60.750909,0.112719,550,China-Shanghai,China,Shanghai,30000000,0.607708
9,52.924800,627,23640.092057,3594.384766,0.643108,512,53.571659,0.088668,630,France-Paris,France,Paris,12100000,2.970566


##  Panel data

In [15]:
# 创建一个从 NAME_2 到 NAME_0 的映射（字典）
name2_to_name0 = gdf.dropna(subset=['NAME_2']).drop_duplicates(subset=['NAME_2'])[['NAME_2', 'NAME_0']]
name2_to_name0_dict = dict(zip(name2_to_name0['NAME_2'], name2_to_name0['NAME_0']))

# 遍历所有唯一的 NAME_2（排除缺失）
# for name in gdf['NAME_2'].dropna().unique():
#     country = name2_to_name0_dict.get(name)
#     print(f"City: {name}, Country: {country}")

results = []

for name in gdf['NAME_2'].unique():
    
    country = name2_to_name0_dict.get(name)
    
    filename = "cough_aggregates_" + name.replace(" ", "_") + "_2024.csv"
    filepath = os.path.join(data_dir, filename)

    df = pd.read_csv(filepath)
    
    # 转换为 datetime 类型
    df['date'] = pd.to_datetime(df['Date_Created'], errors='coerce')
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['cough_rate'] = pd.to_numeric(df['cough_rate'], errors='coerce')
    avg_users = df['total_users'].mean()
    
    if avg_users> 0:  #全部城市
        
        
        filename = name.replace(" ", "-") + "-air-quality.csv"
        air_dir = r"E:\sleep cycle\data\air_quality_all"
        filepath = os.path.join(air_dir, filename)
        df_air = pd.read_csv(filepath)
        
        df_air.columns = df_air.columns.str.strip()
        df_air['date'] = pd.to_datetime(df_air['date'], errors='coerce')
        df_air['date'] = df_air['date'].dt.strftime('%Y-%m-%d')
        df_air['pm25'] = pd.to_numeric(df_air['pm25'], errors='coerce')
        df_air = df_air[['date', 'pm25']]
        
        df_merged = pd.merge(df, df_air, on='date', how='inner')
        df_merged['city'] = name
        df_merged['country'] = country
        
        
        
        filename = name + "_noaa.csv"
        weather_dir = r"E:\sleep cycle\data\weather_all"
        filepath = os.path.join(weather_dir, filename)
        df_wea = pd.read_csv(filepath)
        
        df_wea.columns = df_wea.columns.str.strip()
        df_wea['date'] = pd.to_datetime(df_wea['DATE'], errors='coerce')
        df_wea['date'] = df_wea['date'].dt.strftime('%Y-%m-%d')
        df_wea['TAVG'] = pd.to_numeric(df_wea['TAVG'], errors='coerce').fillna(0)
        df_wea['PRCP'] = pd.to_numeric(df_wea['PRCP'], errors='coerce').fillna(0)
        df_wea = df_wea[['date', 'TAVG','PRCP']]
        df_merged = pd.merge(df_merged, df_wea, on='date', how='inner')
        
        
        results.append(df_merged)
        
df_panel = pd.concat(results, ignore_index=True) 
df_panel = df_panel[['cough_rate','date','pm25','city','country','TAVG','PRCP']]

In [16]:
df_panel


,cough_rate,date,pm25,city,country,TAVG,PRCP
0,0.554037,2023-10-01,79.0,Bangalore,India,75.0,0.02
1,0.873328,2023-10-02,79.0,Bangalore,India,76.0,0.00
2,0.343569,2023-10-03,79.0,Bangalore,India,77.0,0.00
3,0.306689,2023-10-04,79.0,Bangalore,India,77.0,0.00
4,0.458007,2023-10-05,79.0,Bangalore,India,79.0,0.00
...,...,...,...,...,...,...,...
22921,0.548732,2025-02-24,131.0,Zhengzhou,China,42.0,0.00
22922,0.647141,2025-02-25,150.0,Zhengzhou,China,46.0,0.00
22923,0.467985,2025-02-26,140.0,Zhengzhou,China,51.0,0.00
22924,0.738677,2025-02-27,149.0,Zhengzhou,China,50.0,0.00


In [17]:
# df_panel.to_csv('panel_data_all.csv', index=False)

## Add flu rate

In [18]:
import pandas as pd
from scipy.stats import pearsonr
import os

# ✅ 读取主面板数据
df_flu = df_panel.copy()
df_flu["date"] = pd.to_datetime(df_flu["date"])

# ✅ 保存合并后的结果
merged_dfs = []

for country in df_flu["country"].unique():
    try:
        flu_file = f"E:\\sleep cycle\\data\\flu\\flu_daily_{country}.csv"
        if not os.path.exists(flu_file):
            print(f"⚠️ 跳过：未找到 {flu_file}")
            continue

        # 读取该国家的流感数据
        df_flu_country = pd.read_csv(flu_file)
        df_flu_country["date"] = pd.to_datetime(df_flu_country["date"])
        df_flu_country["country"] = country  # 添加字段，便于 merge

        # 合并到主数据中
        df_sub = df_flu[df_flu["country"] == country]
        df_merged = pd.merge(df_sub, df_flu_country[["date", "country", "flu_rate"]],
                             on=["date", "country"], how="left")

        merged_dfs.append(df_merged)

    except Exception as e:
        print(f"❌ 合并 {country} 失败：{e}")

# ✅ 合并所有国家数据
df_flu_final = pd.concat(merged_dfs, ignore_index=True)

# ✅ 保存结果
# df_flu_final.to_csv("panel_data_all_with_flu_rate.csv", index=False)
print("✅ 成功！保存为 df_flu_with_flu_rate.csv")


✅ 成功！保存为 df_flu_with_flu_rate.csv


In [19]:
df_flu_final

,cough_rate,date,pm25,city,country,TAVG,PRCP,flu_rate
0,0.554037,2023-10-01,79.0,Bangalore,India,75.0,0.02,0.134831
1,0.873328,2023-10-02,79.0,Bangalore,India,76.0,0.00,0.142442
2,0.343569,2023-10-03,79.0,Bangalore,India,77.0,0.00,0.142442
3,0.306689,2023-10-04,79.0,Bangalore,India,77.0,0.00,0.142442
4,0.458007,2023-10-05,79.0,Bangalore,India,79.0,0.00,0.142442
...,...,...,...,...,...,...,...,...
22935,0.530524,2025-02-24,42.0,Sao Paulo,Brazil,77.0,0.00,0.025316
22936,0.521488,2025-02-25,49.0,Sao Paulo,Brazil,78.0,0.00,0.025316
22937,0.517456,2025-02-26,49.0,Sao Paulo,Brazil,78.0,0.00,0.025316
22938,0.509430,2025-02-27,58.0,Sao Paulo,Brazil,78.0,0.00,0.025316
